# Wastewater-Based Epidemiology Analysis with KinBiont.jl

This notebook demonstrates the application of KinBiont.jl for Wastewater-Based Epidemiology (WBE) analysis. It covers sampling strategies, time series analysis, and bacterial/viral detection methods.

In [ ]:
# Import required packages
using Pkg
Pkg.activate(".")

# Add KinBiont if not already installed
if !haskey(Pkg.project().dependencies, "KinBiont")
    Pkg.add(path="../../")
end

# Import other necessary packages
using KinBiont
using Plots
using CSV
using DataFrames
using Statistics
using Dates

## 1. Sampling Strategy Optimization

WBE relies on effective sampling strategies to detect pathogens in wastewater. Here we'll explore optimal sampling approaches based on microbial kinetics.

### 1.1 Sampling frequency determination

Determining optimal sampling frequency based on pathogen decay rates and expected population dynamics.

In [ ]:
# Example: Modeling viral decay in wastewater samples
# Parameters for a first-order decay model
decay_rate = 0.2  # Example decay rate per day
initial_concentration = 1000.0  # Initial viral concentration
days = 0:0.1:10

# Simple exponential decay model
concentration = [initial_concentration * exp(-decay_rate * t) for t in days]

# Plot decay curve
plot(days, concentration, 
     label="Viral concentration", 
     xlabel="Time (days)", 
     ylabel="Concentration",
     title="Viral decay in wastewater")

### 1.2 Spatial sampling optimization

Evaluating different sampling locations in a wastewater network to maximize detection probability.

In [ ]:
# Simulate concentration at different sampling points
sampling_points = ["Treatment Plant", "Main Collector", "Residential Zone A", "Residential Zone B", "Industrial Zone"]
mean_concentrations = [100, 250, 400, 180, 50]
variability = [0.2, 0.3, 0.4, 0.25, 0.15]  # Coefficient of variation

# Generate random samples with appropriate variability
using Random
Random.seed!(123)

samples = DataFrame()
for (i, point) in enumerate(sampling_points)
    # Simulate 10 samples from each location with appropriate variability
    conc = mean_concentrations[i] .* (1 .+ variability[i] .* randn(10))
    samples[!, Symbol(point)] = max.(0, conc)  # Ensure no negative concentrations
end

# Calculate detection probability (assuming detection limit of 50)
detection_limit = 50
detection_prob = [mean(col .> detection_limit) for col in eachcol(samples)]

# Plot detection probabilities
bar(sampling_points, detection_prob, 
    label="Detection probability", 
    title="Pathogen Detection Probability by Location",
    ylabel="Probability",
    rotation=45)

## 2. Time Series Analysis Applications

Analyzing temporal patterns in wastewater data using KinBiont's time series capabilities.

### 2.1 Change Point Detection for Outbreak Identification

In [ ]:
# Simulate a time series with an outbreak event
days = 1:100
baseline = 10 .+ 2 .* sin.(2π .* days ./ 7)  # Weekly cycles
outbreak = zeros(100)
outbreak[40:60] = 30 .* exp.(-0.2 .* abs.(collect(40:60) .- 50))  # Outbreak peak at day 50

# Add noise
signal = baseline + outbreak + 3 .* randn(100)
signal = max.(0, signal)  # No negative concentrations

# Apply KinBiont's change point detection
# Create sample data structure
time_data = collect(days)
wbe_data = KinBiont.structure_data_kinbiont("WBE_sample", time_data, signal)

# Apply change point detection
change_points = KinBiont.cpd_function(wbe_data.time, wbe_data.signal, "PELT")

# Plot the results
p = plot(wbe_data.time, wbe_data.signal, 
        label="WBE signal", 
        title="Change Point Detection for Outbreak Identification",
        xlabel="Days", 
        ylabel="Viral concentration")

# Add vertical lines for change points
for cp in change_points
    vline!([wbe_data.time[cp]], label="", color=:red, linestyle=:dash)
end

display(p)

### 2.2 Trend Analysis and Forecasting

In [ ]:
# Create training and test datasets
train_period = 1:80
test_period = 81:100

train_data = KinBiont.structure_data_kinbiont("WBE_train", time_data[train_period], signal[train_period])

# Fit a logistic growth model - useful for modeling disease spread
model_str = "logistic"
time_range = collect(train_data.time[1]:0.1:100)  # For prediction

# Fit model to training data
fitted_params = KinBiont.NL_fit_one_well(train_data, model_str, false)

# Generate predictions
predictions = KinBiont.NL_model_functions(model_str, fitted_params, time_range)

# Plot original data and predictions
p = plot(train_data.time, train_data.signal, label="Training data", marker=:circle)
plot!(time_data[test_period], signal[test_period], label="Test data", marker=:circle)
plot!(time_range, predictions, label="Prediction", lw=2)
xlabel!("Days")
ylabel!("Viral load")
title!("WBE Trend Analysis and Forecasting")

display(p)

## 3. Bacterial Analysis Methods

Applying KinBiont's microbial kinetics models to analyze bacterial populations in wastewater.

### 3.1 Modeling Bacterial Growth Dynamics

In [ ]:
# Simulate bacterial growth with competition using ODE system
# Two bacterial populations competing for resources

function competitive_growth!(du, u, p, t)
    # u[1], u[2]: bacterial populations
    # u[3]: resource
    r1, r2, K1, K2, c1, c2 = p
    
    # Competitive growth equations
    du[1] = r1 * u[1] * (1 - u[1]/K1 - c1*u[2]/K1) * u[3]/(u[3] + 10)
    du[2] = r2 * u[2] * (1 - u[2]/K2 - c2*u[1]/K2) * u[3]/(u[3] + 5)
    du[3] = -0.1 * (du[1] + du[2])  # Resource consumption
end

# Initial conditions and parameters
u0 = [10.0, 5.0, 100.0]  # Initial populations and resource
tspan = (0.0, 100.0)
p = [0.1, 0.08, 1000.0, 800.0, 0.8, 0.6]  # Growth parameters

# Solve the ODE system
prob = KinBiont.ODEProblem(competitive_growth!, u0, tspan, p)
sol = KinBiont.solve(prob, KinBiont.Tsit5())

# Plot results
times = 0:1:100
p1 = plot(sol.t, [sol[1, i] for i in 1:length(sol.t)], 
         label="Bacteria 1", lw=2, 
         title="Bacterial Population Dynamics",
         xlabel="Time (hours)", 
         ylabel="Population")
plot!(sol.t, [sol[2, i] for i in 1:length(sol.t)], 
      label="Bacteria 2", lw=2)

p2 = plot(sol.t, [sol[3, i] for i in 1:length(sol.t)], 
         label="Resource", lw=2, 
         title="Resource Depletion",
         xlabel="Time (hours)", 
         ylabel="Resource Concentration")

plot(p1, p2, layout=(2,1), size=(800, 600))

### 3.2 Indicator Bacteria Analysis for Fecal Contamination

In [ ]:
# Simulate multiple indicator bacteria concentration with decay and regrowth
function indicator_dynamics!(du, u, p, t)
    # u[1]: E. coli, u[2]: Enterococci
    r_growth1, r_growth2, K1, K2, decay1, decay2, temp_effect = p
    
    # Temperature effect (simulating diurnal variation)
    temp = 20 + 5 * sin(2π * t / 24)  # Daily temperature cycle
    temp_factor = exp(temp_effect * (temp - 20) / 10)  # Q10 temperature scaling
    
    # Population dynamics with growth and decay
    du[1] = r_growth1 * u[1] * (1 - u[1]/K1) * temp_factor - decay1 * u[1]
    du[2] = r_growth2 * u[2] * (1 - u[2]/K2) * temp_factor - decay2 * u[2]
end

# Initial conditions and parameters
u0 = [1000.0, 500.0]  # Initial indicators (CFU/100mL)
tspan = (0.0, 120.0)  # 5 days in hours
p = [0.05, 0.03, 10000.0, 5000.0, 0.1, 0.08, 0.8]  # Growth/decay parameters

# Solve the system
prob = KinBiont.ODEProblem(indicator_dynamics!, u0, tspan, p)
sol = KinBiont.solve(prob, KinBiont.Tsit5())

# Plot results
p1 = plot(sol.t, [sol[1, i] for i in 1:length(sol.t)], 
         label="E. coli", lw=2, 
         title="Indicator Bacteria Dynamics",
         xlabel="Time (hours)", 
         ylabel="Concentration (CFU/100mL)",
         yscale=:log10)  # Log scale for better visualization

plot!(sol.t, [sol[2, i] for i in 1:length(sol.t)], 
      label="Enterococci", lw=2)

# Add regulatory limit lines
hline!([235], linestyle=:dash, color=:red, label="E. coli limit")
hline!([70], linestyle=:dash, color=:orange, label="Enterococci limit")

display(p1)

## 4. Integrating Multiple Data Sources for WBE

Combining wastewater data with other epidemiological data sources.

In [ ]:
# Simulate WBE data alongside clinical case data
days = 1:100
clinical_delay = 7  # Clinical cases lag behind wastewater signals

# Create base epidemic curve with wastewater signal
base_signal = 5 .+ zeros(100)
base_signal[30:80] .+= 50 .* exp.(-0.01 .* ((30:80) .- 55).^2)  # Gaussian outbreak

# Add noise to wastewater signal
ww_signal = base_signal .+ 3 .* randn(100)
ww_signal = max.(0, ww_signal)  # No negative values

# Create clinical cases with delay
clinical_cases = zeros(100)
clinical_cases[1:(100-clinical_delay)] = base_signal[(clinical_delay+1):100] ./ 5  # Scaled and delayed
clinical_cases .+= 1 .* randn(100)
clinical_cases = round.(max.(0, clinical_cases))  # Integer counts, no negatives

# Plot comparison
p = plot(days, ww_signal, 
        label="Wastewater viral RNA", 
        xlabel="Day", 
        ylabel="Concentration")

# Add clinical cases on secondary y-axis
p2 = twinx(p)
plot!(p2, days, clinical_cases, 
      label="Clinical cases", 
      color=:red, 
      ylabel="Number of cases")

title!("Comparison of Wastewater Signal and Clinical Cases")
display(p)

## 5. Machine Learning for WBE Pattern Recognition

Using KinBiont's ML capabilities for pattern recognition in wastewater data.

In [ ]:
# Generate synthetic training data
n_samples = 200
features = DataFrame()

# Features: viral concentration, temp, pH, flow rate, turbidity
features.viral_conc = max.(0, 100 .+ 50 .* randn(n_samples))
features.temp = 15 .+ 5 .* randn(n_samples)
features.pH = 7 .+ 0.5 .* randn(n_samples)
features.flow_rate = max.(0, 50 .+ 10 .* randn(n_samples))
features.turbidity = max.(0, 5 .+ 2 .* randn(n_samples))

# Target: outbreak status (1 = yes, 0 = no)
# Define a simple rule: viral_conc > 150 AND temp < 18 increases outbreak probability
p_outbreak = 0.1 .+ 0.8 .* (features.viral_conc .> 150) .* (features.temp .< 18)
outbreak = rand(n_samples) .< p_outbreak

# Train a decision tree model using KinBiont's ML functionality
X = Matrix(features)
y = Int.(outbreak)

# Split data into training and testing sets
train_idx = rand(1:n_samples, Int(floor(0.7 * n_samples)))
test_idx = setdiff(1:n_samples, train_idx)

X_train = X[train_idx, :]
y_train = y[train_idx]
X_test = X[test_idx, :]
y_test = y[test_idx]

# Train decision tree model
model = KinBiont.train_decision_tree(X_train, y_train, names(features), ["No Outbreak", "Outbreak"])

# Predict on test set
predictions = KinBiont.predict_decision_tree(model, X_test)

# Calculate accuracy
accuracy = sum(predictions .== y_test) / length(y_test)
println("Model accuracy: $(round(accuracy * 100, digits=2))%")

# Visualize decision tree
KinBiont.visualize_decision_tree(model, names(features), ["No Outbreak", "Outbreak"])

## 6. Conclusion and Next Steps

This notebook has demonstrated several applications of KinBiont.jl for Wastewater-Based Epidemiology:

1. **Sampling Strategy Optimization**:
   - Determining optimal sampling frequency based on pathogen decay rates
   - Spatial sampling optimization to maximize detection probability

2. **Time Series Analysis**:
   - Change point detection for outbreak identification
   - Trend analysis and forecasting of pathogen concentrations

3. **Bacterial Analysis Methods**:
   - Modeling bacterial growth dynamics in wastewater systems
   - Analysis of indicator bacteria for fecal contamination

4. **Multi-source Data Integration**:
   - Combining wastewater data with clinical case data
   - Evaluating leading indicator properties of WBE data

5. **Machine Learning Applications**:
   - Pattern recognition for outbreak prediction
   - Feature importance analysis for WBE signals

### Next Steps

- Apply these methods to real wastewater surveillance data
- Integrate additional data sources (meteorological, mobility, etc.)
- Develop more sophisticated predictive models
- Validate findings against clinical case data